# HackThisAI: medium_flying_pig

In [ ]:
import matplotlib.pyplot as plt
import os
import subprocess
import time
from PIL import Image
import requests
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.models import resnet50

In [ ]:
%%capture
!git clone https://github.com/JosephTLucas/HackThisAI

In [ ]:
%%capture
!docker build --tag flying_pigs HackThisAI/challenge/medium_flying_pigs/

In [ ]:
%%capture
subprocess.call("docker run -p 5000:5000 flying_pigs:latest &", shell=True)

In [ ]:
print("Waiting to make sure container starts.")
time.sleep(10)

We want to submit a mask, that when added to the provided pig picture, gets the pig classified as an airliner.

In [ ]:
def get_tensor_from_img(img_path):
    img = Image.open(img_path)
    preprocess = transforms.Compose([transforms.Resize([224,224]), transforms.ToTensor()])
    return preprocess(img)[None, :, :, :]

src_dir = "img/"

Let's start with some naive solutions. What happens if we just submit an airliner? Let's go to google images and find one.

![airliner.jpg](img/airliner.jpg)

I chose this specific plane because it had several features that I thought might be important to the classifier including being able to see both wings, being able to see the tail, and being able to see the jet engines.

In [ ]:
plane = get_tensor_from_img(src_dir+"airliner.jpg")
save_image(plane, src_dir+"plane.jpg")

So now we have a plane resized and transformed to be a suitable mask for the pig. What is this amalgamation going to look like to the classifier?

In [ ]:
pig_plane = get_tensor_from_img(src_dir+"pig.jpg") + get_tensor_from_img(src_dir+"plane.jpg")
plt.imshow(pig_plane[0].numpy().transpose(1,2,0))
torch.save(pig_plane, src_dir+"pig_plane.pt")

Notice the clipping warning. We didn't even check to make sure that values remained in the expected range for colors `[0:255]`. I think we should submit it anyway.

In [ ]:
def submit(path):
    with open(path, "rb") as f:
        r = requests.post("http://localhost:5000/check", files={"data_file": f})
    return r.text

submit(src_dir+"pig_plane.pt")

Okay, that didn't work. Remember, we were supposed to submit a mask. The image we see above isn't what's getting submitted to the classifier. It's getting added to the original pig image first. What if we take our plane image and subtract the pig. We know exactly what pig image they're using.

In [ ]:
plane_minus_pig = get_tensor_from_img(src_dir+"plane.jpg") - get_tensor_from_img(src_dir+"pig.jpg")
plt.imshow(plane_minus_pig[0].numpy().transpose(1,2,0))
torch.save(plane_minus_pig, src_dir+"plane_minus_pig.pt")

Again, we didn't bother dealing with colors out of the expected bounds, but we see the negative pig and our positive plane. It's almost art. Let's submit that.

In [ ]:
submit(src_dir+"plane_minus_pig.pt")

## BONUS

Nice! But as we can see above, that wouldn't fool a human. It _really_ won't fool a human once we add the positive pig back in... it's just a plane. In most adversarial ML discussions, we're really interested in the _minimal_ adversarial examples. Using our own pretrained resnet, let's build an offline example and a minimal adversarial example using this guide: https://adversarial-ml-tutorial.org/introduction/

In [ ]:
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]


norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:
%%capture
model = resnet50(pretrained=True)
model.eval()

In [ ]:
pig = get_tensor_from_img(src_dir+"pig.jpg")
delta = torch.zeros_like(pig, requires_grad=True)
opt = optim.SGD([delta], lr=5e-3)
epsilon = 2./255

for t in range(100):
    pred = model(norm(pig + delta))
    # 341 corresponds to hog, 404 to airliner
    loss = (-nn.CrossEntropyLoss()(pred, torch.LongTensor([341])) + 
            nn.CrossEntropyLoss()(pred, torch.LongTensor([404])))
    
    opt.zero_grad()
    loss.backward()
    opt.step()
    delta.data.clamp_(-epsilon, epsilon)

In [ ]:
torch.save(delta, src_dir+"delta.pt")

Let's save `delta` and submit it.

In [ ]:
submit(src_dir+"delta.pt")

As we can see below, we've really made _very_ little change to the image.

In [ ]:
delta.mean()

In [ ]:
plt.imshow(delta[0].detach().numpy().transpose(1,2,0))

In [ ]:
print("MODIFIED")
plt.imshow((pig+delta)[0].detach().numpy().transpose(1,2,0))

In [ ]:
print("ORIGINAL")
plt.imshow((pig)[0].detach().numpy().transpose(1,2,0))